# EKF short example 

From: https://docs.google.com/presentation/d/1fT3nua171rOZXKoyEC3YevsTVMiCbaPVEXpt2vZPRho/edit?usp=drive_link

![](data/car.png)

## State and input vectors
$ \mathbf{X} =  \begin{bmatrix}
        p \\
        \dot{p} \\
     \end{bmatrix} \quad  \quad \mathbf{u} = \ddot{p} $

## Motion/Process model

$ \mathbf{X_k = f(\hat{X}_{k-1}, u_{k-1}, w_{k-1})} =  \begin{bmatrix}
        1 & dt \\
        0 & 1 \\
     \end{bmatrix} \mathbf{X_{k-1}}  + 
     \begin{bmatrix}
        0 \\
        dt \\
     \end{bmatrix} \mathbf{u}_{k-1} +  \mathbf{w_{k-1}},  \quad \quad 
    \mathbf{w_k} \sim N(\mathbf{0}, (0.1) \mathbf{I}_2) $

## Linearized motion model 

$ \mathbf{X_k =  f(\hat{X}_{k-1}, u_{k-1}, 0)  
       + F_{k-1} (X_{k-1} - \hat{X}_{k-1}) 
       + L_{k-1} w_{k-1}} $


Jacobian Matrix:  $ \mathbf{F_{k-1} =  \frac{\partial{f}}{\partial{x_{k-1}}}(\hat{X}_{k-1}, u_{k-1}, 0)
       = \begin{bmatrix}
              1 & dt \\
              0 & 1 \\
       \end{bmatrix}} $

Jacobian Matrix: $ \mathbf{L_{k-1} = \frac{\partial{f}}{\partial{w_{k-1}}}(\hat{X}_{k-1}, u_{k-1}, 0) =  
       \begin{bmatrix}
              1 & 0 \\
              0 & 1 \\
       \end{bmatrix}} $



## Landmark measurement model 

$ y_k = \phi_k  = h(p_k, v_k) = tan^{-1}(\frac{S}{D-p_k}) + v_k, \quad \quad \quad  v_k \sim N(0, 0.01)  $

## Linearized measurement model 

$ \mathbf{y_k = h_k(\breve{x}_k, 0) + H_k(x_k - \breve{x}_k) + M_k v_k }  $

Jacobian Matrix:  $ \mathbf{H_k =  \frac{\partial{h}}{\partial{x_k}}(\breve{x}_k, 0)
       = \begin{bmatrix}
              \frac{S}{(D-\breve{p}_k)^{2} + S^2} & 0 \\
       \end{bmatrix}} $

Jacobian Matrix:  $ \mathbf{M_k =  \frac{\partial{h}}{\partial{x_k}}(\breve{x}_k, 0) = 1} $

In [1]:
import numpy as np

x0_hat = np.array([ [0, 5] ]).T
P0_hat = np.array([[0.01, 0], [0, 1]])

dt = 0.5
u = -2

y1 = np.pi/6 

S = 20 
D = 40 

F = np.array([[1, dt], [0, 1]])
L = np.identity(2)

G = np.array([[0], [dt]])
Q = 0.1 * np.identity(2)

R = 0.01*np.identity(1)

def f(x, u, w):
    return np.dot(F, x) + np.dot(G, u) + w

def h(x):
    return np.arctan2(S, D-x[0, 0])

def Hk(x):
    pk_check = x[0, 0] 
    denom = (D - pk_check)*(D - pk_check) + S*S
    hh = np.zeros( (1, 2))
    hh[0, 0] = S/denom
    return hh


# Prediction step

$ \mathbf{ \breve{x}_k = f_{k-1}(\hat{X}_{k-1}, u_{k-1}, 0) }$

$ \mathbf{ \breve{P}_k = F_{k-1} \hat{P}_{k-1} F^{T}_{k-1}  + L_{k-1} Q_{k-1} L^{T}_{k-1} }$


In [ ]:
w0 = np.zeros((2, 1))
x1_check = f(x0_hat, u, w0)
print("predicted state: \n", x1_check)

P1_check = np.dot(F, P0_hat).dot(F.T) + np.dot(L, Q).dot(L.T)
print("predicted covariance: \n", P1_check)

predicted state: 
 [[2.5]
 [4. ]]
predicted covariance: 
 [[0.36 0.5 ]
 [0.5  1.1 ]]


# Measurement update step 

Optimal gain: $ \mathbf{ K_k = \breve{P}_k H^{T}_{k} ( H_k \breve{P}_k H^{T}_{k} + M_k R_k M^T_k})^{-1} $

$ \mathbf{ \hat{X}_{k} = \breve{x}_k + K_k (y_k - h_k(\breve{x}_k, 0)) }$


$ \mathbf{ \hat{P}_k = (I - K_k H_k) \breve{P}_k }$


In [ ]:
from numpy.linalg import inv

H = Hk(x1_check)
M = np.identity(1)
K = np.dot(P1_check, H.T).dot( inv(np.dot(H, P1_check).dot(H.T) + np.dot(M, R).dot(M.T)))
print("K = \n", K)

res = y1 - h(x1_check)

x1_hat = x1_check  + np.dot(K, res)
print("x1_hat = \n", x1_hat)

P1_hat = (np.identity(2) - np.dot(K, H)).dot(P1_check)
print("P1_hat = \n", P1_hat)

K = 
 [[0.39686426]
 [0.55120036]]
x1_hat = 
 [[2.51335109]
 [4.01854318]]
P1_hat = 
 [[0.35841804 0.49780283]
 [0.49780283 1.09694837]]
